In [4]:
from gad_adversarial_robustness.gad.dominant import dominant
from pygod.utils import load_data
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from sklearn.metrics import roc_auc_score
import os
from gad_adversarial_robustness.utils.graph_utils import prepare_graph
from torch_geometric.data import Data
import numpy as np
from gad_adversarial_robustness.utils.graph_utils import get_n_anomaly_indexes
from typing import List
from gad_adversarial_robustness.poison.greedy import multiple_AS

script_dir = os.path.abspath('')


dataset_caching_path = os.path.join(script_dir, '..', '..', '..', 'data')

## run the poison

In [5]:




import copy
from gad_adversarial_robustness.poison.greedy import poison_attack


clean_data: Data = load_data("inj_cora", dataset_caching_path)
poisoned_data: Data = load_data("inj_cora", dataset_caching_path)

truth: List[int] = poisoned_data.y.bool()
amount_of_nodes = poisoned_data.num_nodes
_, adj, _ = prepare_graph(poisoned_data)
dense_adj = adj.to_dense()  #Fill in zeroes where there are no edges


print("Create poison compatible adjacency matrix...")
triple = []
for i in range(amount_of_nodes):
    for j in range(i + 1, amount_of_nodes):
        triple.append([i, j, dense_adj[i,j]])  #Fill with 0, then insert actual after



triple = np.array(triple)

# These are the nodes we try reduce the active subnetwork score for (disguising anonomaly nodes)
target_node_lst = get_n_anomaly_indexes(truth, 999)
#target_node_lst = get_n_anomaly_indexes(truth, 10)
print(type(target_node_lst)), print(f'target node list: {target_node_lst}'), print(target_node_lst)

print("Making model...")
model = multiple_AS(target_lst = target_node_lst, n_node = amount_of_nodes, device = 'cpu')
budget = 100  # The amount of edges to change


print("Starting attack...")
#triple = triple.detach().cpu().numpy()

#triple = torch.tensor(triple, requires_grad=True)


adj_adversary, _, _ = poison_attack(model, triple, budget)

print("Converting to compatible tensor...")

# Create Edge Index'
edge_index = torch.tensor([[],[]])

# Transpose it to make shape compatible
transposed_adj_adversary = torch.transpose(adj_adversary, 0, 1)

for i in range(len(adj_adversary)):
    if(adj_adversary[i][2] != 0):   #If edge value is not 0 (no edge)
        #Add edge to edge index, choosing first 2 elements (edges), and then the ith edge
        edge_index = torch.cat((edge_index, transposed_adj_adversary[:2, i:i+1]), -1)
        # Dataset uses edges both ways so add reverse edge as well
        edge_index = torch.cat((edge_index, torch.flip(transposed_adj_adversary[:2, i:i+1], dims=[0])), -1)


edge_index = edge_index.type(torch.int64)
poisoned_data.edge_index = edge_index



tensor([1., 1., 1.,  ..., 1., 1., 1.])
node_attrs
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
adj
SparseTensor(row=tensor([   0,    0,    0,  ..., 2707, 2707, 2707]),
             col=tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706]),
             val=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
             size=(2708, 2708), nnz=11054, density=0.15%)
labels
tensor([0, 0, 0,  ..., 0, 0, 0])
Create poison compatible adjacency matrix...


KeyboardInterrupt: 

In [ ]:
import yaml
from gad_adversarial_robustness.gad.dominant.dominant import Dominant
from gad_adversarial_robustness.utils.graph_utils import load_anomaly_detection_dataset

yaml_path = os.path.join(script_dir, '..', 'configs', 'dominant_config.yaml')
with open(yaml_path) as file:
        config = yaml.safe_load(file)

"""
Dominant on clean data
"""

adj, attrs, label, adj_label = load_anomaly_detection_dataset(clean_data)
adj = torch.FloatTensor(adj)
adj_label = torch.FloatTensor(adj_label)
attrs = torch.FloatTensor(attrs)


model = Dominant(feat_size=attrs.size(1), hidden_size=config['hidden_dim'], dropout=config['dropout'], 
                 device=config['device'], adj=adj, adj_label=adj_label, attrs=attrs, label=label)
model.fit(config)

"""
Dominant on poisoned data
"""

adj, attrs, label, adj_label = load_anomaly_detection_dataset(poisoned_data)
adj = torch.FloatTensor(adj)
adj_label = torch.FloatTensor(adj_label)
attrs = torch.FloatTensor(attrs)


model = Dominant(feat_size=attrs.size(1), hidden_size=config['hidden_dim'], dropout=config['dropout'], 
                 device=config['device'], adj=adj, adj_label=adj_label, attrs=attrs, label=label)
model.fit(config)



Epoch: 0099, Auc: 0.8400129701686123
Epoch: 0099, Auc: 0.8388823098178538
